# Notebook for hyperparameter tuning

This is a very simple notebook to test out some algorithms for hyperparameter tuning, to find one that works well and apply it to our model

In [5]:
# Necessary imports

import sys
sys.path.append("../")

from Hack import load 
from Hack import rl
import matplotlib.pyplot as plt
import numpy as np
import optuna
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env

%matplotlib qt5

In [2]:
# load epex data
epex = load.epex().load()
price_array = epex['apx_da_hourly'].values

start_of_2020 = None
start_of_2021 = None

for idx, (i, row) in enumerate(epex.iterrows()):
    if i.year > 2019 and start_of_2020 is None:
        start_of_2020 = idx
    if i.year > 2020 and start_of_2021 is None:
        start_of_2021 = idx
        break

In [10]:
def objective():
    """
    Function to take in hyperparameters, train a reinforcement model, and output the "profit" of the model as a metric
    """

    start_idx = 0
    end_idx = start_of_2020 #4 * 2*24*7 #start_of_2020 # 2019->2020 # 2*24*7
    obs_price_array = price_array[start_idx:end_idx]

    power = 0.5
    env = rl.energy_price_env(obs_price_array, window_size=24*2, power=power)
    model = PPO(MlpPolicy, env, verbose=0)      
    model.learn(total_timesteps = 50000)

    test_start_idx = start_of_2020
    test_end_idx =  start_of_2021

    test_price_array = price_array[test_start_idx:test_end_idx]

    new_env =  DummyVecEnv([lambda: rl.energy_price_env(test_price_array, power=power)])
    mean_reward_after_train = rl.evaluate(model, new_env=new_env, num_episodes=100, index=epex.index[test_start_idx:test_end_idx])

    return mean_reward_after_train

SyntaxError: invalid syntax (1556052740.py, line 13)